# 0.0. INIT

## 0.1. Imports

In [5]:
import pandas as pd
import psycopg2 as pg
from credentials import CREDENTIALS

## 0.2. Helper functions

In [6]:
def connect_db():
    credentials = CREDENTIALS

    return pg.connect(**credentials)

def query_db(query: str):
    conn = connect_db()
    cursor = conn.cursor()


    cursor.execute(query)
    record = cursor.fetchall()

    cursor.close()
    conn.close()

    return record

# 1.0. POSTGRES DATABASE

## 1.1. Table Query

In [7]:
query_tables = """
    select tablename
    from pg_tables
    where schemaname = 'pa004'
"""

query_db(query_tables)

[('users',), ('vehicle',), ('insurance',)]

## 1.2. Data Query

In [6]:
query_schema = """
    select *
    from pa004.users u
    where u.age > 44
    limit 10
"""

record = query_db(query_schema)


In [8]:
pd.DataFrame(record).head()

,0,1,2,3,4
0,2,Male,76,3.0,26.0
1,3,Male,47,28.0,26.0
2,8,Female,56,28.0,26.0
3,11,Female,47,35.0,124.0
4,14,Male,76,28.0,13.0


In [11]:
conn = connect_db()
query_df = """
    select *
    from
        pa004.users u 	inner join pa004.vehicle v 		on(u.id = v.id)
                        inner join pa004.insurance i 	on(u.id = i.id)
"""

df = pd.read_sql(query_df, conn)
conn.close()

In [13]:
df = df.drop(df.columns[[5, 9]], axis=1)
df.head()

,gender,age,region_code,policy_sales_channel,driving_license,vehicle_age,vehicle_damage,previously_insured,annual_premium,vintage,response
0,Male,23,11.0,152.0,1,< 1 Year,Yes,0,23367.0,249,0
1,Female,41,15.0,14.0,1,1-2 Year,No,1,31409.0,221,0
2,Female,25,35.0,152.0,1,< 1 Year,No,1,46622.0,299,0
3,Female,26,8.0,160.0,1,< 1 Year,No,0,2630.0,136,0
4,Male,45,8.0,124.0,1,1-2 Year,Yes,0,42297.0,264,0


In [15]:
df.to_csv('../data/raw/data_raw.csv', index=False)